# Bechdel Analysis
#### CS5850 Final Project

Alex Thurston, Hailey Dennis, Tyler Kunz

# Data Collection and Cleaning

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from numpy import linalg,mean,dot
from pandas import DataFrame
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Get five-thirty-eight bechdel data
def get_five_thirty_eight_bechdel_data():
    data = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/bechdel/movies.csv")
    select_columns = data[['year', 'imdb', 'title', 'clean_test', 'binary', 'budget_2013$', 'domgross_2013$', 'intgross_2013$']]
    return select_columns

In [3]:
# Print correlation and covariance matrices
def print_corr_cov_matrices(data):
    print(data.corr())
    print(data.cov())

In [4]:
# Check for duplicates: True if duplicates were found, False if all data is unique
def check_duplicates(data):
    if len(select_columns['imdb']) != len(select_columns['imdb'].unique()):
        return True
    else:
        return False

In [5]:
# Add column to dataframe
def add_column(df, col_name, new_list):
    df.insert(len(df.columns), str(col_name), new_list)
    return df

In [6]:
# Returns percentage of women in given crew role (e.g. Producer, Director
def get_perc_women_role(crew_response, job_type):
    female_count = 0
    job_count = 0
    for person in crew_response['crew']:
        if person['job'] == job_type:
            job_count += 1
            if person['gender'] == 1:
                female_count += 1
    if job_count == 0:
        return None
    else:
        return female_count / job_count

In [ ]:
# Populate data
data_file_name = "bechdel_analysis_data.csv"
data_file_path = "./bechdel_analysis_data.csv"

# See if data has already been populated in the current directory
if os.path.isfile(data_file_path):
    select_columns = pd.read_csv(data_file_path)
else:
    select_columns = get_five_thirty_eight_bechdel_data()
    
    #Gather TMDB data - setup
    tmdb_api_key = "55d7071c3daf17bcf8cc0f4a6f688a24"
    movie_ids = select_columns['imdb']


    # Initialize empty arrays
    adult = [None] * len(movie_ids)
    genres = [None] * len(movie_ids)
    prod_comps = [None] * len(movie_ids)
    overview = [None] * len(movie_ids)
    perc_women_producers = [None] * len(movie_ids)
    perc_women_directors = [None] * len(movie_ids)

    # Query the API for data
    for i in range(len(movie_ids)):
        try:
            response = requests.get("https://api.themoviedb.org/3/movie/" + str(movie_ids[i]) + "?api_key=" + str(tmdb_api_key) + "&language=en-US").json()
            crew_response = requests.get("https://api.themoviedb.org/3/movie/" + str(movie_ids[i]) + "/credits?api_key=" + str(tmdb_api_key) + "&language=en-US").json()
            adult[i] = response['adult']
            genres[i] = ','.join([ genre['name'] for genre in response['genres']])
            prod_comps[i] = ','.join([company['name'] for company in response['production_companies']])
            overview[i] = response['overview']
            perc_women_producers[i] = get_perc_women_role(crew_response, 'Producer')
            perc_women_directors[i] = get_perc_women_role(crew_response, 'Director')

        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}")
            continue

    # Add desired columns to select_columns
    desired_cols = {'adult': adult, 'genres': genres, 'prod_comps': prod_comps, 'overview': overview,
                    'perc_women_producers': perc_women_producers, 
                    'perc_women_directors': perc_women_directors}
    for key in desired_cols.keys():
        select_columns = add_column(select_columns, key, desired_cols[key])

    #Save to csv so we don't have to requery the API every time
    select_columns.to_csv(data_file_name, encoding='utf-8', index=False)

Unexpected err=KeyError('adult'), type(err)=<class 'KeyError'>
Unexpected err=KeyError('adult'), type(err)=<class 'KeyError'>
Unexpected err=KeyError('adult'), type(err)=<class 'KeyError'>
Unexpected err=KeyError('adult'), type(err)=<class 'KeyError'>
Unexpected err=KeyError('adult'), type(err)=<class 'KeyError'>
Unexpected err=KeyError('adult'), type(err)=<class 'KeyError'>


In [ ]:
# Check for null values: return columns and sums of null values located in each
def null_values_in_columns(data):
    null_cols = data.isnull().any()
    return [data.columns[i] for i in range(len(data.columns)) if null_cols[i] == True]

cols_with_nulls = null_values_in_columns(select_columns)
print(cols_with_nulls)

In [ ]:
# mutative function to remove rows with nulls
def get_non_null_rows(df, col_with_nulls):
    df = df[df[col_with_nulls].isnull() == False]
    return df

In [ ]:
for i in cols_with_nulls:
    select_columns = get_non_null_rows(select_columns, i)
    
select_columns.dropna()

In [ ]:
print("Null values in columns")
print(null_values_in_columns(select_columns))
print()
print("Null counts")
print(select_columns.isnull().sum())

In [ ]:
import json
import string
import re
import nltk
import preprocessor as pre
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

In [ ]:
string.punctuation

In [ ]:
def clean_text(text):
    no_escape = re.sub(r'&.+?;', '', text)
    no_punc = "".join([i for i in no_escape if i not in string.punctuation])
    lower_text = no_punc.lower()
    return lower_text
    

In [ ]:
# Clean text for later use
select_columns['title'] = select_columns['title'].apply(clean_text)
select_columns['overview'] = select_columns['overview'].apply(clean_text)
select_columns

In [ ]:
select_columns.to_csv("bechdel_analysis_data_cleaned.csv", encoding='utf-8', index=False)

# Data Preprocessing / Feature Selection
Determining correlation and variance among and between features, summarizing, and looking at individual correlation with Bechdel test results.

In [ ]:
def currency_conversion(amt):
    return "${:,.2f}".format(amt)

# Modes = 'overall', 'passing', 'failing' -- make sure you pass in data that corresponds to the mode!
def quick_metrics(column_name, xdata, mode, currency=False):
    if currency:
        print(f"{column_name} min {mode}:", currency_conversion(xdata.min()))
        print(f"{column_name} max {mode}:", currency_conversion(xdata.max()))
        print(f"{column_name} mean {mode}:", currency_conversion(xdata.mean()))
        print(f"{column_name} standard deviation {mode}:", currency_conversion(xdata.std()))
    else:
        print(f"{column_name} min {mode}:", xdata.min())
        print(f"{column_name} max {mode}:", xdata.max())
        print(f"{column_name} mean {mode}:", xdata.mean())
        print(f"{column_name} standard deviation {mode}:", xdata.std())

In [ ]:
data = pd.read_csv("bechdel_analysis_data_cleaned.csv", float_precision=None)
data = data.dropna()
data

In [ ]:
data.corr()

In [ ]:
data.cov()

Convert the pass/fail string binary column to 1=pass and 0=fail.

In [ ]:
pass_fail_data = pd.get_dummies(data.binary, prefix='binary')
pass_fail_data = pass_fail_data.drop("binary_FAIL", axis=1)
data = data.drop(columns=['binary'])
data.insert(5, "binary_PASS", pass_fail_data["binary_PASS"])
data_new = data
data_new

In [ ]:
data_new.corr()

In [ ]:
data_new.mad()

Observations:
- Low positive correlation between year and passing
- Low negative correlation between all monetary categories and passing
- Fairly high correlations between all monetary categories with each other
- Percentage of women directors had the largest correlation (positive) with passing at .2
- High correlation between domgross_2013 and int_gross2013
 - We will opt to remove int_gross2013 as it has a significantly lower mean absolute deviation
 - We will keep budget_2013 as it stil offers significant variability
- Adult needs to be removed

### Domestic Gross Analysis

In [ ]:
data_fail = data_new.loc[data_new["binary_PASS"] == 0] 
data_pass = data_new.loc[data_new["binary_PASS"] == 1] 

In [ ]:
quick_metrics('domgross_2013$', data_new['domgross_2013$'], 'overall', True)

In [ ]:
quick_metrics('domgross_2013$', data_pass['domgross_2013$'], 'passing', True)

In [ ]:
quick_metrics('domgross_2013$', data_fail['domgross_2013$'], 'failing', True)

In [ ]:
plt.scatter(data_new["binary_PASS"], data_new["domgross_2013$"])
plt.title("Passing Movies versus Domestic Gross Sales")
plt.xlabel("Binary_PASS: 0 - Failed, 1 - Passed")
plt.ylabel("Domestic Gross Sales ($100 million)")

In [ ]:
plt.bar(["Pass", "Fail"], [data_pass["domgross_2013$"].mean(), data_fail["domgross_2013$"].mean()])
plt.title("Bechdel Test Result vs. Average Domestic Gross")
plt.xlabel("Bechdel Test Result")
plt.ylabel("2013 Average Domestic Gross ($100 million)")

Observations:

- There is a clear correlation between 2013 average domestic gross and Bechdel Test result
- Lower domestic gross sales means higher likelihood to pass the Bechdel test


### Budget Analysis

In [ ]:
quick_metrics('budget_2013$', data_new['budget_2013$'], 'overall', True)

In [ ]:
quick_metrics('budget_2013$', data_pass['budget_2013$'], 'passing', True)

In [ ]:
quick_metrics('budget_2013$', data_fail['budget_2013$'], 'failing', True)

In [ ]:
plt.scatter(data_new["binary_PASS"], data_new["budget_2013$"])
plt.title("Passing Movies versus Budget")
plt.xlabel("Binary_PASS: 0 - Failed, 1 - Passed")
plt.ylabel("Budget ($100 million)")

In [ ]:
plt.bar(["Pass", "Fail"], [data_pass["domgross_2013$"].mean(), data_fail["domgross_2013$"].mean()])
plt.title("Bechdel Test Result vs. Average Budget")
plt.xlabel("Bechdel Test Result")
plt.ylabel("Average Budget ($100 million)")

Observations:

- Similar observations to the domestic gross sales
- Movies that fail the Bechdel test have a higher average budget

### Genre Analysis
We use one-hot-encoding to represent genres as they have useful predictive power. We will not be doing this for production companies as they don't have as much predictive power and there are significantly more.

In [ ]:
data_new["genres"] = data_new["genres"].str.split(',')

In [ ]:
mlb = MultiLabelBinarizer()
genre_data = pd.DataFrame(mlb.fit_transform(data_new["genres"]),
                          columns=mlb.classes_,
                          index=data_new.index)
genre_data

In [ ]:
main_data_genre_data_combined = pd.concat([data_new.reset_index(drop=True),genre_data.reset_index(drop=True)], axis=1)
main_data_genre_data_combined = main_data_genre_data_combined.drop(['genres'], axis=1)
main_data_genre_data_combined.to_csv("bechdel_analysis_data_cleaned_with_genres.csv")
main_data_genre_data_combined

In [ ]:
genre_data.to_csv("genre_data.csv")

In [ ]:
print("Genre by Movie Count")
genre_data.sum().sort_values(ascending=False)

In [ ]:
pass_fail_genre = pd.concat([pass_fail_data, genre_data], axis=1)

In [ ]:
print("Correlation between genre and test results")
pass_fail_genre_corr = pass_fail_genre.corr()
pass_fail_genre_corr[["binary_PASS"]]

In [ ]:
print("Covariance between genre and test results")
pass_fail_genre_cov = pass_fail_genre.cov()
pass_fail_genre_cov[["binary_PASS"]]

- Genres with a clear positive correlation above 0.1 include romance and horror
- Genres with a clear negative correlation above 0.1 include action, adventure, and crime

### Year Analysis

In [ ]:
year_data = data_new[['binary_PASS','year']].groupby('year').count()
plt.plot(year_data.index,year_data['binary_PASS'])
plt.title("Movies Per Year In Database")
plt.ylabel("Count")
plt.xlabel("Year")

In [ ]:
print("\nPass rate by year")
year_pass_data = data_new[data_new["binary_PASS"] == 1]
year_pass_data = year_pass_data[['binary_PASS','year']].groupby('year').count()
pass_percentage_data = year_pass_data / year_data
pass_percentage_data = pass_percentage_data.fillna(0)
pass_percentage_data

In [ ]:
plt.plot(pass_percentage_data.index,pass_percentage_data['binary_PASS'])
plt.title("Proportion of Movies Passing the Bechdel Test by Year")
plt.ylabel("Pass Proportion")
plt.xlabel("Year")

In [ ]:
pass_percentage_post94 = pass_percentage_data[pass_percentage_data.index >= 1994].mean()
pass_percentage_pre94 = pass_percentage_data[pass_percentage_data.index < 1994].mean()


print("Average pass rate for movies prior to 1994:\n", pass_percentage_pre94)
print("Average pass rate for movies 1994 and later:\n", pass_percentage_post94)

- Pass rate fluctuated greatly by year in early years, likely because of the smaller sample size
- Rates seem to be stabilizing around .4 to .5 over time
- Average pass rate slowly increases over time
- Number of movies per year spiked following 1994-ish
- Majority of our data averages near a 40-50% pass rate, but is lower prior to 1994 (less values)

### Adult Movie Analysis

In [ ]:
adult_data = pd.get_dummies(data.adult, prefix='adult')
adult_data

No adult movies in the dataset.

### Production Company Analysis

We cover this in the Data Storage section.

In [ ]:
data_new["prod_comps"] = data_new["prod_comps"].str.split(',')
data_new

### Percentage of Women Producers Analysis

In [ ]:
# Percentage of movies with no women producers
no_women = data_new.loc[data_new['perc_women_producers'] == 0]
no_women.shape[0] / data_new.shape[0]

In [ ]:
quick_metrics("perc_women_producers", data_new['perc_women_producers'], 'overall', False)

In [ ]:
quick_metrics("perc_women_producers", data_pass['perc_women_producers'], 'passing', False)

In [ ]:
quick_metrics("perc_women_producers", data_fail['perc_women_producers'], 'failing', False)

In [ ]:
p_percs = data_new['perc_women_producers'].unique()
p_percs.sort()

pass_p_percs = []

for p in p_percs:
    rows = data_new.loc[data_new['perc_women_producers'] == p]
    passed = rows.loc[data_new['binary_PASS'] == 1]
    pass_p_percs.append(passed.shape[0] / rows.shape[0])
    
plt.plot(p_percs, pass_p_percs)
plt.title("Pass rates by Percentage of Producers")
plt.xlabel("Percentage of Women Producers")
plt.ylabel("Percentage Passing")

### Percentage of Women Directors Analysis

In [ ]:
# Percentage of movies with no women directors
no_women = data_new.loc[data_new['perc_women_directors'] == 0]
no_women.shape[0] / data_new.shape[0]

In [ ]:
quick_metrics("perc_women_directors", data_new['perc_women_directors'], 'overall', False)

In [ ]:
quick_metrics("perc_women_directors", data_pass['perc_women_directors'], 'passing', False)

In [ ]:
quick_metrics("perc_women_directors", data_fail['perc_women_directors'], 'failing', False)

In [ ]:
d_percs = data_new['perc_women_directors'].unique()
# d_percs = d_percs[1:]
d_percs.sort()

pass_d_percs = []

for p in d_percs:
    rows = data_new.loc[data_new['perc_women_directors'] == p]
    passed = rows.loc[data_new['binary_PASS'] == 1]
    pass_d_percs.append(passed.shape[0] / rows.shape[0])
    
plt.plot(d_percs, pass_d_percs)
plt.title("Pass rates by Percentage of Women Directors")
plt.xlabel("Percentage of Women Directors")
plt.ylabel("Percentage Passing")

### Observations on Crew Gender

- Having women in director roles is much more indicative of a passed test than in producer roles
- It is important to note that most of the movies have no women in these roles, so the majority of the data sits at 0%
- The mean percentage of women in production roles overall is 16.15%
- The mean percentage of women in director roles overall is 7.1%
- Graphs are limited by differences in movie count per percentage
- Regardless, having women in producer and director roles leads to a higher average pass rate

### Test Passing Binary Analysis

In [ ]:
plt.bar(["Pass", "Fail"], [data_pass.shape[0], data_fail.shape[0]])
plt.title("Overall Movie Count by Test Result")
plt.xlabel("Test Result")
plt.ylabel("Movie Count")

We have more failing movies than passing in our dataset overall.

### Principal Component Analysis

In [ ]:
pca_data = data_new[["year", "binary_PASS","budget_2013$","domgross_2013$"]]
pca_data

In [ ]:
numComponents = 2
cov = pca_data.cov().to_numpy()
A = pca_data.to_numpy()
[eigvals, pcs] = linalg.eig(cov)

sorted_index = np.argsort(eigvals)[::-1]
eigvals = eigvals[sorted_index]
pcs = pcs[:,sorted_index]

print(eigvals)
M = (A-mean(A.T,axis=1)).T
projected = dot(pcs.T,M).T
projected = DataFrame(projected[:,:numComponents],columns=['pc1','pc2'])
projected

In [ ]:
projected.plot(kind='scatter',x='pc1',y='pc2')

In [ ]:
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(pca_data.to_numpy())
data_rescaled.shape


pca = PCA().fit(data_rescaled)


plt.rcParams["figure.figsize"] = (12,6)

fig, ax = plt.subplots()
xi = np.arange(1, 5, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 5, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

In [ ]:
pcs[:,:numComponents]

In [ ]:
from pandas import Series

fig,axes = plt.subplots(2,1,sharex=True)
attrib = list(pca_data)         # get attribute names
pcdata  = Series(pcs[:,0], index=attrib)
pcdata.plot(kind='barh',ax=axes[0],color='k',alpha=0.7)
axes[0].set_title(r'1st PC', size = 'x-large')
pcdata  = Series(pcs[:,1], index=attrib)
pcdata.plot(kind='barh',ax=axes[1],color='k',alpha=0.7)
axes[1].set_title(r'2nd PC', size = 'x-large')

Observations:
- Our PCA results show a best combination of the domestic gross sales and budget.
- We feel that we gain better comprehension by separating these, and will opt not to use the PCs.

### Summary

- We will remove the adult column as all values are false.
- We will change the representation of our classification column (binary test pass/fail to 0 or 1)
- We will remove intgross_2013 as it correlates highly with domgross_2013 but has a lower mean absolute deviation.
- The rest of our dataset has highly variable features.

In [ ]:
data = data.drop(['adult'], axis=1)
data = data.drop(['intgross_2013$'], axis=1)
data.to_csv("bechdel_analysis_data_cleaned.csv", encoding='utf-8', index=False)

# Data Storage / SQL

In the context of this project, we are using MySQL to perform queries for further data summarization and visualization. Our dataset is small enough that we feel comfortable keeping it in CSV form for other analytics.

In [ ]:
import mysql.connector
from mysql.connector import errorcode
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt

### Database Setup
This portion populates a MySQL server with values from a pandas DataFrame.

In [ ]:
# Run this the first time
# !pip install pymysql

In [ ]:
# Configure to your own user and password
mysql_user = 'root'
mysql_pw = ''
mysql_host = 'localhost'

engine = sqlalchemy.create_engine(f"mysql+pymysql://{mysql_user}:{mysql_pw}@{mysql_host}")
conn = engine.connect()
conn

In [ ]:
data = pd.read_csv("bechdel_analysis_data_cleaned.csv", float_precision=None)

In [ ]:
def execute_dml(dml_string):
    try:
        conn.execute(dml_string)
    except mysql.connector.Error as err:
        print("Error:", err)
        
def execute_query(query_string):
    try:
        res = conn.execute(query_string)
        return res.fetchall()
    except mysql.connector.Error as err:
        print("Error:", err)

def create_db():
    drop_existing_db = "DROP DATABASE IF EXISTS bechdel;"
    create_bechdel = "CREATE DATABASE bechdel;"
    use_bechdel = "USE bechdel;"
    try:
        execute_dml(drop_existing_db)
        execute_dml(create_bechdel)
        execute_dml(use_bechdel)
    except mysql.connector.Error as err:
        print("Error:", err)
        
def populate_db():
    general_data = data.drop(['genres', 'prod_comps'], axis=1)
    general_data.to_sql('general', conn, if_exists='replace', dtype={
        'year': sqlalchemy.types.INTEGER(),
        'imdb': sqlalchemy.types.VARCHAR(length=12),
        'title': sqlalchemy.types.VARCHAR(length=255),
        'clean_test': sqlalchemy.types.VARCHAR(length=20),
        'binary_passed': sqlalchemy.types.INTEGER(),
        'budget_2013$': sqlalchemy.types.BIGINT(),
        'domgross_2013$': sqlalchemy.types.BIGINT(),
        'intgross_2013$': sqlalchemy.types.BIGINT(),
        'overview': sqlalchemy.types.TEXT(),
        'perc_women_producers': sqlalchemy.types.FLOAT(),
        'perc_women_directors': sqlalchemy.types.FLOAT()
    })
    
    # Convert into workable list type before using .explode()
    genre_data = data[['imdb', 'genres']]
    for i in range(genre_data.count()[0]):
        genre_data.iloc[i].genres = genre_data.iloc[i].genres.strip('[]').replace('\'', "").split(', ')
    genre_data = genre_data.explode('genres')
    genre_data.to_sql('genre', conn, if_exists='replace', dtype={
        'imdb': sqlalchemy.types.VARCHAR(length=12),
        'genre': sqlalchemy.types.VARCHAR(length=20)
    })
    
    prod_data = data[['imdb', 'prod_comps']]
    for i in range(prod_data.count()[0]):
        prod_data.iloc[i].prod_comps = prod_data.iloc[i].prod_comps.strip('[]').replace('\'', "").split(', ')
    prod_data = prod_data.explode('prod_comps')
    prod_data.to_sql('production_company', conn, if_exists='replace', dtype={
        'imdb': sqlalchemy.types.VARCHAR(length=12),
        'genre': sqlalchemy.types.VARCHAR(length=50)
    })
    execute_dml('ALTER TABLE general ADD PRIMARY KEY (imdb);')
    execute_dml('ALTER TABLE genre ADD FOREIGN KEY (imdb) REFERENCES general(imdb);')
    execute_dml('ALTER TABLE production_company ADD FOREIGN KEY (imdb) REFERENCES general(imdb);')

In [ ]:
create_db()
populate_db()

print(execute_query("DESC general"), '\n');
print(execute_query("DESC genre"), '\n');
print(execute_query("DESC production_company"));

### Querying the Database

#### Overall Financial Analysis

In [ ]:
print('Avg budget, domestic gross sales, and internationaly gross sales for movies that passed')

res = execute_query("""
    SELECT CONCAT('$', FORMAT(avg(budget_2013$), 2)),
           CONCAT('$', FORMAT(avg(domgross_2013$), 2))
        FROM general
    WHERE binary_PASS = 1;
"""
)
print(res[0])

print('\nAvg budget, domestic gross sales, and internationaly gross sales for movies that failed')
res = execute_query("""
    SELECT CONCAT('$', FORMAT(avg(budget_2013$), 2)),
           CONCAT('$', FORMAT(avg(domgross_2013$), 2))
        FROM general
    WHERE binary_PASS = 0;
"""
)
print(res[0])

In [ ]:
res = execute_query("""
    SELECT title,
           domgross_2013$,
           binary_PASS
    FROM general g
    ORDER BY domgross_2013$ DESC
    LIMIT 50;
""")
highest_sales = pd.DataFrame(res)
print("\nResults for 50 highest selling movies in the dataset")
highest_sales

In [ ]:
res = execute_query("""
    SELECT title,
           budget_2013$,
           binary_PASS
    FROM general g
    ORDER BY budget_2013$ DESC
    LIMIT 50;
""")
highest_budget = pd.DataFrame(res)
print("\nResults for 50 highest budget movies in the dataset")
highest_budget

#### Production Company Analysis

In [ ]:
res = execute_query("""
    SELECT COUNT(*) AS movie_count,
           prod_comps,
           SUM(CASE WHEN g.binary_PASS = 1 THEN 1 ELSE 0 END) / COUNT(*) AS perc_passing
    FROM production_company pc
    JOIN general g ON pc.imdb = g.imdb
    GROUP BY prod_comps
    ORDER BY movie_count DESC
    LIMIT 100;
""")
prod_passing = pd.DataFrame(res)
print("\nPassing percentages for most popular production companies in dataset\n")
prod_passing

In [ ]:
plt.plot(prod_passing.movie_count, prod_passing.perc_passing)
plt.title("Passing Percentage by Production Company Movie Count")
plt.xlabel("Movies Produced by Company")
plt.ylabel("Percent of Movies Passing Bechdel Test")

In [ ]:
res = execute_query("""
    SELECT COUNT(*) AS movie_count,
           prod_comps,
           SUM(CASE WHEN g.binary_PASS = 1 THEN 1 ELSE 0 END) / COUNT(*) AS perc_passing
    FROM production_company pc
    JOIN general g ON pc.imdb = g.imdb
    GROUP BY prod_comps
    HAVING perc_passing = 1
    ORDER BY movie_count DESC;
""")
prod_passing = pd.DataFrame(res)
print("Production companies with 100% pass rate, by movie count\n")
prod_passing

#### Genre Analysis

In [ ]:
res = execute_query("""
    SELECT COUNT(*) AS movie_count,
           genres,
           SUM(CASE WHEN g.binary_PASS = 1 THEN 1 ELSE 0 END) / COUNT(*) AS perc_passing
    FROM genre gn
    JOIN general g ON gn.imdb = g.imdb
    GROUP BY genres
    ORDER BY movie_count DESC;
""")
genre_passing = pd.DataFrame(res)
print("Passing percentages by genre\n")
genre_passing.sort_values(by=['perc_passing'], ascending=False)

#### Clean_test Analysis
Note that only values of 'ok' indicate a pass in the binary_PASS column

In [ ]:
res = execute_query("""
    SELECT COUNT(*) AS movie_count,
           clean_test
    FROM general g
    GROUP BY clean_test
    ORDER BY movie_count DESC;
""")
clean_test_freq = pd.DataFrame(res)
print("Frequencies for clean_test classifications\n")
clean_test_freq.sort_values(by=['movie_count'], ascending=False)

### Overall Observations
* The average budget and sales are higher for movies that fail the Bechdel test.
* The majority of movies in the top 50 for highest budget and sales fail the Bechdel test.
* The most popular production companies in our data set (by movie count) mostly have passing percentages under 50%.
* Production companies with 100% pass rate have a maximum of 6 movies in our data set.
* As you increase the count of movies by production company, the pass rate stabilizes around 40-50%'
* Romance, Horror, and Music are the most likely to pass the test in our data set, though these all have very different movie counts.
* Much of our data set passes one or more criterion, but most do not pass all 3.

# Text Analysis
The purpose of this section is to gather information on the text frequencies and predictive power based on movie titles and overviews.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
data = pd.read_csv("bechdel_analysis_data_cleaned.csv", float_precision=None)
data

In [ ]:
vectorizer = CountVectorizer()
overviews_bag = vectorizer.fit_transform(data["overview"])
titles_bag = vectorizer.fit_transform(data["title"])

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
type(overviews_bag.toarray())
overviews_bag.toarray()[0]

#### Prediction Using Bag of Words on Movie Overview

In [ ]:

X = overviews_bag.toarray()
y = data["binary_PASS"]

X_train, X_test, y_train, y_test = train_test_split(X, y)
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter=300)

In [ ]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)


In [ ]:
metrics.accuracy_score(y_test, y_pred)

#### Prediction TF-IDF on Movie Overview

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(data["overview"])
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head())
len(df)

In [ ]:
X = tfIdf.toarray()
y = data["binary_PASS"]

X_train, X_test, y_train, y_test = train_test_split(X, y)
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')

In [ ]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

#### Prediction Using Bag of Words on Movie Title

In [ ]:
X = titles_bag.toarray()
y = data["binary_PASS"]

X_train, X_test, y_train, y_test = train_test_split(X, y)
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')

In [ ]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

#### Prediction Using TF-IDF on Movie Title

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(data["title"])
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head())


In [ ]:
X = tfIdf.toarray()
y = data["binary_PASS"]

X_train, X_test, y_train, y_test = train_test_split(X, y)
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter=100)

In [ ]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

#### Misra Gries

In [ ]:
def misra_gries(token, token_dict, k):
    if token in token_dict:
        token_dict[token] += 1
    elif len(token_dict) < k - 1:
        token_dict[token] = 1
    else:
        for token in token_dict.copy():
            token_dict[token] -= 1
            if token_dict[token] == 0:
                token_dict.pop(token)
    return token_dict

In [ ]:
def calc_false_pos(file_name, k, freqs):
    fp = open(file_name, 'r')
    false_pos = []
    token_dict = {}
    token_count = 0
    while True:
        token = fp.readline().strip()
        if not token:
            break
        token_count += 1
        if token in token_dict:
            token_dict[token] += 1
        else:
            token_dict[token] = 1
    for token in freqs:
        if token_dict[token] <= (token_count/k):
            false_pos.append(token)
    fp.close()
    print("TOKEN COUNT", token_count)
    print("Proportion of false positives:", (len(false_pos)/len(freqs)))
    return false_pos       

In [ ]:
titles_tokenized = data_pass.title.str.split()
overviews_tokenized = data_pass.overview.str.split()

with open('passing_title_tokens.txt', 'w') as f:    
    for row in titles_tokenized:
        for word in row:
            try:
                f.write(word)
                f.write('\n')
            except:
                continue
f.close()

with open('passing_overview_tokens.txt', 'w') as f:    
    for row in overviews_tokenized:
        for word in row:
            try:
                f.write(word)
                f.write('\n')
            except:
                continue
f.close()

fail_data = data[data["binary_PASS"] == 0]
titles_tokenized = fail_data.title.str.split()
overviews_tokenized = fail_data.overview.str.split()

with open('failing_title_tokens.txt', 'w') as f:    
    for row in titles_tokenized:
        for word in row:
            try:
                f.write(word)
                f.write('\n')
            except:
                continue
f.close()

with open('failing_overview_tokens.txt', 'w') as f:    
    for row in overviews_tokenized:
        for word in row:
            try:
                f.write(word)
                f.write('\n')
            except:
                continue
f.close()

In [ ]:
def run(file_name, k=10):
    fp = open(file_name, 'r')
    token_dict = {}
    token_count = 0
    while True:
        token = fp.readline().strip()
        if not token:
            print("End of the stream, m =", token_count)
            fp.close()
            return token_dict
        token_dict = misra_gries(token, token_dict, k)
        token_count += 1

In [ ]:
pt_freqs = run('passing_title_tokens.txt', 500)
print(pt_freqs)
print()
print("False positives:", calc_false_pos('passing_title_tokens.txt', 15, pt_freqs))

In [ ]:
po_freqs = run('passing_overview_tokens.txt', 1000)
print(po_freqs)
print()
print("False positives:", calc_false_pos('passing_overview_tokens.txt', 15, po_freqs))

In [ ]:
ft_freqs = run('failing_title_tokens.txt', 250)
print(ft_freqs)
print()
print("False positives:", calc_false_pos('failing_title_tokens.txt', 15, ft_freqs))

In [ ]:
fo_freqs = run('failing_overview_tokens.txt', 1500)
print(fo_freqs)
print()
print("False positives:", calc_false_pos('failing_overview_tokens.txt', 15, fo_freqs))

#### Conclusions
- The movie overview was a much better predictor for passing the bechdel test, likely because the overview simply contains more data. Each summary has multiple sentences of data, as opposed to just a few words for the title.
- The TF-IDF approach worked slightly better than the bag of words. 
- Stop words make misra gries difficult, but make for better prediction metrics.
- Misra Gries was ineffective for this data set as there aren't many frequent words besides stop words.

# Predictive Modeling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, tree, neighbors, linear_model, ensemble, naive_bayes, svm, neural_network
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc, mean_squared_error, accuracy_score
import math

In [ ]:
data = pd.read_csv("bechdel_analysis_data_cleaned_with_genres.csv", float_precision=None)
data

In [ ]:
prod_comp_data = data.prod_comps
overview_data = data.overview

data = data.drop(['prod_comps', 'overview', 'title', 'imdb'], axis=1)

In [ ]:
Y = data.binary_PASS
data = data.drop(['binary_PASS', 'clean_test'], axis=1)
X = data

In [ ]:
X

In [ ]:
Y

In [ ]:
# Decision Tree
dt_accuracy = {1: None, 5: None, 10: None, 50: None, 100: None}
for i in dt_accuracy:
    # Train decision tree classifier with given max_depth
    dt_clf = tree.DecisionTreeClassifier(max_depth=i)
    dt_accuracy[i] = cross_val_score(dt_clf, X, Y, cv=5).mean()
    

In [ ]:
# K-nearest Neighbors
knn_accuracy = {1: None, 2: None, 3: None, 4: None, 5: None, 10: None, 15: None}
for i in knn_accuracy:
    # Train k-nearest neigbor classifier with given n_neighbors
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=i)
    knn_accuracy[i] = cross_val_score(knn_clf, X, Y, cv=5).mean()

In [ ]:
# Logistic Regression
lr_accuracy = {0.001: None, 0.01: None, 0.1: None, 0.5: None, 1: None}
for i in lr_accuracy:
    # Train logistic regression classifier with given C
    lr_clf = linear_model.LogisticRegression(C=i)
    lr_accuracy[i] = cross_val_score(lr_clf, X, Y, cv=5).mean()

In [ ]:
# Naive Bayes
nb_accuracy = {0.001: None, 0.01: None, 0.1: None, 0.5: None, 1: None}
for i in nb_accuracy:
    # Train naive bayes classifier with given alpha
    nb_clf = naive_bayes.ComplementNB(alpha=i)
    nb_accuracy[i] = cross_val_score(nb_clf, X, Y, cv=5).mean()

In [ ]:
# Support Vector Machine
sv_accuracy = {0.001: None, 0.01: None, 0.1: None, 0.5: None, 1: None}
for i in sv_accuracy:
    # Train support vector classifier with given C
    sv_clf = svm.SVC(C=i)
    sv_accuracy[i] = cross_val_score(sv_clf, X, Y, cv=5).mean()

In [ ]:
# Multilayer Perceptron Classifier (NN)
mlp_accuracy = {10: None, 25: None, 50: None, 75: None, 100: None}
for i in mlp_accuracy:
    # Train multilayer perceptron classifier with given 
    mlp_clf = neural_network.MLPClassifier(hidden_layer_sizes=(i,))
    mlp_accuracy[i] = cross_val_score(mlp_clf, X, Y, cv=5).mean()

In [ ]:
# Random Forest
rf_accuracy = {10: None, 25: None, 50: None, 100: None, 150: None}
for i in rf_accuracy:
    # Train random forest classifier with given n_estimators
    rf_clf = ensemble.RandomForestClassifier(n_estimators=i)
    rf_accuracy[i] = cross_val_score(rf_clf, X, Y, cv=5).mean()

In [ ]:
plt.figure()
plt.title("Hyperparameter Accuracy for Decision Tree Classifier")
plt.plot()
x = [i for i in dt_accuracy.keys()]
y = [i for i in dt_accuracy.values()]

plt.plot(x,y)
plt.xticks(x)
plt.ylabel("Five-fold Cross-Validation Accuracy")
plt.xlabel("max_depth")
plt.show()

plt.figure()
plt.title("Hyperparameter Accuracy for K-Nearest Neighbors Classifier")
x = [i for i in knn_accuracy.keys()]
y = [i for i in knn_accuracy.values()]

plt.plot(x,y)
plt.xticks(x)
plt.ylabel("Five-fold Cross-Validation Accuracy")
plt.xlabel("n_neighbors")
plt.show()

plt.figure()
plt.title("Hyperparameter Accuracy for Logistic Regression")
x = [i for i in lr_accuracy.keys()]
y = [i for i in lr_accuracy.values()]

plt.plot(x,y)
plt.xticks(x, rotation=45)
plt.ylabel("Five-fold Cross-Validation Accuracy")
plt.xlabel("C")
plt.show()

plt.figure()
plt.title("Hyperparameter Accuracy for Naive Bayes")
x = [i for i in nb_accuracy.keys()]
y = [i for i in nb_accuracy.values()]

plt.plot(x,y)
plt.xticks(x, rotation=45)
plt.ylabel("Five-fold Cross-Validation Accuracy")
plt.xlabel("alpha")
plt.show()

plt.figure()
plt.title("Hyperparameter Accuracy for Support Vector Classifier")
plt.plot()
x = [i for i in sv_accuracy.keys()]
y = [i for i in sv_accuracy.values()]

plt.plot(x,y)
plt.xticks(x)
plt.ylabel("Five-fold Cross-Validation Accuracy")
plt.xlabel("C")
plt.show()

plt.figure()
plt.title("Hyperparameter Accuracy for Multilayer Perceptron Classifier")
plt.plot()
x = [i for i in mlp_accuracy.keys()]
y = [i for i in mlp_accuracy.values()]

plt.plot(x,y)
plt.xticks(x)
plt.ylabel("Five-fold Cross-Validation Accuracy")
plt.xlabel("hidden_layer_sizes")
plt.show()

plt.figure()
plt.title("Hyperparameter Accuracy for Random Forest Classifier")
plt.plot()
x = [i for i in rf_accuracy.keys()]
y = [i for i in rf_accuracy.values()]

plt.plot(x,y)
plt.xticks(x)
plt.ylabel("Five-fold Cross-Validation Accuracy")
plt.xlabel("n_estimators")
plt.show()

In [ ]:
dt_clf = tree.DecisionTreeClassifier(max_depth=1)
dt_clf.fit(X, Y)
tree.plot_tree(dt_clf, feature_names=[i for i in X.columns], class_names='binary_PASS',filled=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

# Train DT
dt_clf = tree.DecisionTreeClassifier(max_depth=1)
dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)

# Train KNN
knn_clf = neighbors.KNeighborsClassifier(n_neighbors=10)
knn_clf.fit(x_train, y_train)
knn_pred = knn_clf.predict(x_test)

# Train LR
lr_clf = linear_model.LogisticRegression(C=0.5)
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)

# Train NB
nb_clf = naive_bayes.ComplementNB(alpha=0.5)
nb_clf.fit(x_train, y_train)
nb_pred = nb_clf.predict(x_test)

# Train SVC
sv_clf = svm.SVC(C=0.5, probability=True)
sv_clf.fit(x_train, y_train)
sv_pred = sv_clf.predict(x_test)

# Train MLP
mlp_clf = neural_network.MLPClassifier(hidden_layer_sizes=(50,))
mlp_clf.fit(x_train, y_train)
mlp_pred = mlp_clf.predict(x_test)

# Train RF
rf_clf = ensemble.RandomForestClassifier(n_estimators=10)
rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)

In [ ]:
# Calculate confusion matrix
dt_conf_mat = confusion_matrix(y_test, dt_pred)
dt_f1 = f1_score(y_test, dt_pred)

knn_conf_mat = confusion_matrix(y_test, knn_pred)
knn_f1 = f1_score(y_test, knn_pred)

lr_conf_mat = confusion_matrix(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)

nb_conf_mat = confusion_matrix(y_test, nb_pred)
nb_f1 = f1_score(y_test, nb_pred)

sv_conf_mat = confusion_matrix(y_test, sv_pred)
sv_f1 = f1_score(y_test, sv_pred)

mlp_conf_mat = confusion_matrix(y_test, mlp_pred)
mlp_f1 = f1_score(y_test, mlp_pred)

rf_conf_mat = confusion_matrix(y_test, rf_pred)
rf_f1 = f1_score(y_test, rf_pred)

# Print output
print("Decision Tree Confusion Matrix")
print(dt_conf_mat)
print("K-Nearest Neighbor Confusion Matrix")
print(knn_conf_mat)
print("Logistic Regression Confusion Matrix")
print(lr_conf_mat)
print("Naive Bayes Confusion Matrix")
print(nb_conf_mat)
print("SVC Confusion Matrix")
print(sv_conf_mat)
print("Multilayer Perceptron Confusion Matrix")
print(mlp_conf_mat)
print("Random Forest Confusion Matrix")
print(rf_conf_mat)

In [ ]:
# Precision, recall for Decision Tree
dt_p = (dt_conf_mat[0][0]/(dt_conf_mat[0][0] + dt_conf_mat[1][0]))
dt_r = (dt_conf_mat[0][0]/(dt_conf_mat[0][0] + dt_conf_mat[0][1]))

# Precision, recall for K-Nearest Neighbor
knn_p = (knn_conf_mat[0][0]/(knn_conf_mat[0][0] + knn_conf_mat[1][0]))
knn_r = (knn_conf_mat[0][0]/(knn_conf_mat[0][0] + knn_conf_mat[0][1]))

# Precision, recall for Logistic Regression
lr_p = (lr_conf_mat[0][0]/(lr_conf_mat[0][0] + lr_conf_mat[1][0]))
lr_r = (lr_conf_mat[0][0]/(lr_conf_mat[0][0] + lr_conf_mat[0][1]))

# Precision, recall for Naive Bayes
nb_p = (nb_conf_mat[0][0]/(nb_conf_mat[0][0] + nb_conf_mat[1][0]))
nb_r = (nb_conf_mat[0][0]/(nb_conf_mat[0][0] + nb_conf_mat[0][1]))

# Precision, recall for Support Vector Machine
sv_p = (sv_conf_mat[0][0]/(sv_conf_mat[0][0] + sv_conf_mat[1][0]))
sv_r = (sv_conf_mat[0][0]/(sv_conf_mat[0][0] + sv_conf_mat[0][1]))

# Precision, recall for Multilayer Perceptron
mlp_p = (mlp_conf_mat[0][0]/(mlp_conf_mat[0][0] + mlp_conf_mat[1][0]))
mlp_r = (mlp_conf_mat[0][0]/(mlp_conf_mat[0][0] + mlp_conf_mat[0][1]))

# Precision, recall for Random Forest Classifier
rf_p = (rf_conf_mat[0][0]/(rf_conf_mat[0][0] + rf_conf_mat[1][0]))
rf_r = (rf_conf_mat[0][0]/(rf_conf_mat[0][0] + rf_conf_mat[0][1]))

In [ ]:
# Generate output
classifiers = ['DT', 'KNN', 'LR', 'NB', 'SVC', 'MLP', 'RF']
precisions = [dt_p, knn_p, lr_p, nb_r, sv_r, mlp_r, rf_r]
recalls = [dt_r, knn_r, lr_r, nb_p, sv_p, mlp_p, rf_p]
f1s = [dt_f1, knn_f1, lr_f1, nb_f1, sv_f1, mlp_f1, rf_f1]
x_axis= np.arange(len(classifiers))

plt.figure()
plt.bar(x_axis-0.2, precisions, width=0.2, label="Precision")
plt.bar(x_axis, recalls, width=0.2, label="Recall")
plt.bar(x_axis+0.2, f1s, width=0.2, label="F1")

plt.xticks(x_axis, classifiers)
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()


In [ ]:
# DT Stats
dt_prob = dt_clf.predict_proba(x_test)
dt_labels = (y_test == 1)

dt_fpr, dt_tpr, dt_thresh = roc_curve(dt_labels, dt_prob[:,1])
dt_roc_auc = auc(dt_fpr, dt_tpr)
dt_legend = "DT (AUC = %0.4f)" % dt_roc_auc

# KNN Stats
knn_prob = knn_clf.predict_proba(x_test)
knn_labels = (y_test == 1)

knn_fpr, knn_tpr, knn_thresh = roc_curve(knn_labels, knn_prob[:,1])
knn_roc_auc = auc(knn_fpr, knn_tpr)
knn_legend = "KNN (AUC = %0.4f)" % knn_roc_auc

# LR Stats
lr_prob = lr_clf.predict_proba(x_test)
lr_labels = (y_test == 1)

lr_fpr, lr_tpr, lr_thresh = roc_curve(lr_labels, lr_prob[:,1])
lr_roc_auc = auc(lr_fpr, lr_tpr)
lr_legend = "LR (AUC = %0.4f)" % lr_roc_auc

# NB Stats
nb_prob = nb_clf.predict_proba(x_test)
nb_labels = (y_test == 1)

nb_fpr, nb_tpr, nb_thresh = roc_curve(nb_labels, nb_prob[:,1])
nb_roc_auc = auc(nb_fpr, nb_tpr)
nb_legend = "NB (AUC = %0.4f)" % nb_roc_auc

# SVC Stats
sv_prob = sv_clf.predict_proba(x_test)
sv_labels = (y_test == 1)

sv_fpr, sv_tpr, sv_thresh = roc_curve(sv_labels, sv_prob[:,1])
sv_roc_auc = auc(sv_fpr, sv_tpr)
sv_legend = "SV (AUC = %0.4f)" % sv_roc_auc

# MLP Stats
mlp_prob = mlp_clf.predict_proba(x_test)
mlp_labels = (y_test == 1)

mlp_fpr, mlp_tpr, mlp_thresh = roc_curve(mlp_labels, mlp_prob[:,1])
mlp_roc_auc = auc(mlp_fpr, mlp_tpr)
mlp_legend = "MLP (AUC = %0.4f)" % mlp_roc_auc

# SVC Stats
rf_prob = rf_clf.predict_proba(x_test)
rf_labels = (y_test == 1)

rf_fpr, rf_tpr, rf_thresh = roc_curve(rf_labels, rf_prob[:,1])
rf_roc_auc = auc(rf_fpr, rf_tpr)
rf_legend = "RF (AUC = %0.4f)" % rf_roc_auc

In [ ]:
# Make AUC plot
plt.figure()

plt.plot(dt_fpr, dt_tpr, label=dt_legend)
plt.plot(knn_fpr, knn_tpr, label=knn_legend)
plt.plot(lr_fpr, lr_tpr, label=lr_legend)
plt.plot(nb_fpr, nb_tpr, label=nb_legend)
plt.plot(sv_fpr, sv_tpr, label=sv_legend)
plt.plot(mlp_fpr, mlp_tpr, label=mlp_legend)
plt.plot(rf_fpr, rf_tpr, label=rf_legend)

plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

## Interesting Observations
The models are all around the level of a random guess at this point. Logistic Regression is the best here, but not by much. From here, we revisted from preprocessing and only got slightly better results. We next try a grid search to help tune hyperparameters.

## Brute Force - GridSearch

In [ ]:
# Decision Tree
dt_params = {'criterion': ['gini', 'entropy'],'splitter': ['best', 'random'], 'max_depth': [None, 1, 5, 10, 50], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 5, 10]}

dt = tree.DecisionTreeClassifier()

dt_grid_search = GridSearchCV(estimator=dt, param_grid=dt_params, cv=5, n_jobs=-1,
                          verbose=1, scoring='accuracy')

dt_grid_search.fit(X, Y)

dt_grid_search.best_estimator_

In [ ]:
# K-Nearest Neighbors
knn_params = {'n_neighbors': [1, 3, 5, 10, 50], 
              'weights': ['uniform', 'distance'], 
              'algorithm': ['ball_tree', 'kd_tree', 'brute'], 
              'leaf_size': [5, 10, 30, 50]}

knn = neighbors.KNeighborsClassifier()

knn_grid_search = GridSearchCV(estimator=knn, param_grid=knn_params, cv=5, n_jobs=-1,
                          verbose=1, scoring='accuracy')

knn_grid_search.fit(X, Y)

knn_grid_search.best_estimator_

In [ ]:
# Logisitic Regression
lr_params = {'C': [0.001, 0.01, 0.1, 0.5, 1], 
              'solver': ['liblinear', 'lbfgs', 'newton-cholesky'], 
              'penalty': ['l1', 'l2', 'elasticnet', None]}

lr = linear_model.LogisticRegression()

lr_grid_search = GridSearchCV(estimator=lr, param_grid=lr_params, cv=5, n_jobs=-1,
                          verbose=1, scoring='accuracy')

lr_grid_search.fit(X, Y)

lr_grid_search.best_estimator_

In [ ]:
# Naive Bayes
nb_params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 1.5], 
              'norm': [True, False]}


nb = naive_bayes.ComplementNB()

nb_grid_search = GridSearchCV(estimator=nb, param_grid=nb_params, cv=5, n_jobs=-1,
                          verbose=1, scoring='accuracy')

nb_grid_search.fit(X, Y)

nb_grid_search.best_estimator_

In [ ]:
# Commented out for convenience as the search takes significantly longer than other models
# Support Vector Machine
# sv_params = {'C': [0.001, 0.01, 0.1, 0.5, 1], 
#               'kernel': ['linear', 'poly', 'rbf']}
# # , 
# #               'gamma': ['scale', 'auto']}


# sv = svm.SVC(probability=True)

# sv_grid_search = GridSearchCV(estimator=sv, param_grid=sv_params, cv=5, n_jobs=-1,
#                           verbose=1, scoring='accuracy')

# sv_grid_search.fit(X, Y)

# sv_grid_search.best_estimator_

In [ ]:
# Multilayer Perceptron
mlp_params = {'hidden_layer_sizes': [(10,), (25,), (50,), (100,)],
              'activation': ['logistic', 'relu', 'identity'],
              'solver': ['lbfgs', 'sgd', 'adam'],
              'learning_rate': ['constant', 'adaptive'],
              'alpha': [0.0001, 0.001, 0.01]}

mlp = neural_network.MLPClassifier()

mlp_grid_search = GridSearchCV(estimator=mlp, param_grid=mlp_params, cv=5, n_jobs=-1,
                          verbose=1, scoring='accuracy')

mlp_grid_search.fit(X, Y)

mlp_grid_search.best_estimator_

In [ ]:
# Random Forest
params = {'n_estimators': [10, 25, 50, 100], 
          'criterion': ['gini', 'entropy'],
          'max_depth': [None, 1, 5, 10, 50],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 5, 10]}

rf = ensemble.RandomForestClassifier()

rf_grid_search = GridSearchCV(estimator=rf, param_grid=params, cv=5, n_jobs=-1,
                          verbose=1, scoring='accuracy')

rf_grid_search.fit(X, Y)

rf_grid_search.best_estimator_

In [ ]:
# Decision Tree
# Predict labels
dt_best = dt_grid_search.best_estimator_
dt_pred = dt_best.predict(x_test)

# Confusion matrix and F1 score
dt_conf_mat = confusion_matrix(y_test, dt_pred)
dt_f1 = f1_score(y_test, dt_pred)

# Precision and recall
dt_p = (dt_conf_mat[0][0]/(dt_conf_mat[0][0] + dt_conf_mat[1][0]))
dt_r = (dt_conf_mat[0][0]/(dt_conf_mat[0][0] + dt_conf_mat[0][1]))

# DT stats
dt_prob = dt_best.predict_proba(x_test)
dt_labels = (y_test == 1)

dt_fpr, dt_tpr, dt_thresh = roc_curve(dt_labels, dt_prob[:,1])
dt_roc_auc = auc(dt_fpr, dt_tpr)
dt_legend = "DT (AUC = %0.4f)" % dt_roc_auc

# K-Nearest Neighbors
# Predict labels
knn_best = knn_grid_search.best_estimator_
knn_pred = knn_best.predict(x_test)

# Confusion matrix and F1 score
knn_conf_mat = confusion_matrix(y_test, knn_pred)
knn_f1 = f1_score(y_test, knn_pred)

# Precision and recall
knn_p = (knn_conf_mat[0][0]/(knn_conf_mat[0][0] + knn_conf_mat[1][0]))
knn_r = (knn_conf_mat[0][0]/(knn_conf_mat[0][0] + knn_conf_mat[0][1]))

# KNN stats
knn_prob = knn_best.predict_proba(x_test)
knn_labels = (y_test == 1)

knn_fpr, knn_tpr, knn_thresh = roc_curve(knn_labels, knn_prob[:,1])
knn_roc_auc = auc(knn_fpr, knn_tpr)
knn_legend = "KNN (AUC = %0.4f)" % knn_roc_auc

# Logistic Regression
# Predict labels
lr_best = lr_grid_search.best_estimator_
lr_pred = lr_best.predict(x_test)

# Confusion matrix and F1 score
lr_conf_mat = confusion_matrix(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)

# Precision and recall
lr_p = (lr_conf_mat[0][0]/(lr_conf_mat[0][0] + lr_conf_mat[1][0]))
lr_r = (lr_conf_mat[0][0]/(lr_conf_mat[0][0] + lr_conf_mat[0][1]))

# LR stats
lr_prob = lr_best.predict_proba(x_test)
lr_labels = (y_test == 1)

lr_fpr, lr_tpr, lr_thresh = roc_curve(lr_labels, lr_prob[:,1])
lr_roc_auc = auc(lr_fpr, lr_tpr)
lr_legend = "LR (AUC = %0.4f)" % lr_roc_auc

# Naive Bayes
# Predict labels
nb_best = nb_grid_search.best_estimator_
nb_pred = nb_best.predict(x_test)

# Confusion matrix and F1 score
nb_conf_mat = confusion_matrix(y_test, nb_pred)
nb_f1 = f1_score(y_test, nb_pred)

# Precision and recall
nb_p = (nb_conf_mat[0][0]/(nb_conf_mat[0][0] + nb_conf_mat[1][0]))
nb_r = (nb_conf_mat[0][0]/(nb_conf_mat[0][0] + nb_conf_mat[0][1]))

# NB stats
nb_prob = nb_best.predict_proba(x_test)
nb_labels = (y_test == 1)

nb_fpr, nb_tpr, nb_thresh = roc_curve(nb_labels, nb_prob[:,1])
nb_roc_auc = auc(nb_fpr, nb_tpr)
nb_legend = "NB (AUC = %0.4f)" % nb_roc_auc

# Multilayer Perceptron
# Predict labels
mlp_best = mlp_grid_search.best_estimator_
mlp_pred = mlp_best.predict(x_test)

# Confusion matrix and F1 score
mlp_conf_mat = confusion_matrix(y_test, mlp_pred)
mlp_f1 = f1_score(y_test, mlp_pred)

# Precision and recall
mlp_p = (mlp_conf_mat[0][0]/(mlp_conf_mat[0][0] + mlp_conf_mat[1][0]))
mlp_r = (mlp_conf_mat[0][0]/(mlp_conf_mat[0][0] + mlp_conf_mat[0][1]))

# MLP stats
mlp_prob = mlp_best.predict_proba(x_test)
mlp_labels = (y_test == 1)

mlp_fpr, mlp_tpr, mlp_thresh = roc_curve(mlp_labels, mlp_prob[:,1])
mlp_roc_auc = auc(mlp_fpr, mlp_tpr)
mlp_legend = "MLP (AUC = %0.4f)" % mlp_roc_auc

# Random Forest
# Predict labels
rf_best = rf_grid_search.best_estimator_
rf_pred = rf_best.predict(x_test)

# Confusion matrix and F1 score
rf_conf_mat = confusion_matrix(y_test, rf_pred)
rf_f1 = f1_score(y_test, rf_pred)

# Precision and recall
rf_p = (rf_conf_mat[0][0]/(rf_conf_mat[0][0] + rf_conf_mat[1][0]))
rf_r = (rf_conf_mat[0][0]/(rf_conf_mat[0][0] + rf_conf_mat[0][1]))

# RF stats
rf_prob = rf_best.predict_proba(x_test)
rf_labels = (y_test == 1)

rf_fpr, rf_tpr, rf_thresh = roc_curve(rf_labels, rf_prob[:,1])
rf_roc_auc = auc(rf_fpr, rf_tpr)
rf_legend = "RF (AUC = %0.4f)" % rf_roc_auc

In [ ]:
# Optimized DT
tree.plot_tree(dt_best, feature_names=[i for i in X.columns], class_names='binary_PASS',filled=True)

In [ ]:
# Generate output
classifiers = ['DT', 'KNN', 'LR', 'NB', 'MLP', 'RF']
precisions = [dt_p, knn_p, lr_p, nb_r, mlp_r, rf_r]
recalls = [dt_r, knn_r, lr_r, nb_p, mlp_p, rf_p]
f1s = [dt_f1, knn_f1, lr_f1, nb_f1, mlp_f1, rf_f1]
x_axis= np.arange(len(classifiers))

plt.figure()
plt.bar(x_axis-0.2, precisions, width=0.2, label="Precision")
plt.bar(x_axis, recalls, width=0.2, label="Recall")
plt.bar(x_axis+0.2, f1s, width=0.2, label="F1")

plt.xticks(x_axis, classifiers)
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()


In [ ]:
# Make AUC plot
plt.figure()

plt.plot(dt_fpr, dt_tpr, label=dt_legend)
plt.plot(knn_fpr, knn_tpr, label=knn_legend)
plt.plot(lr_fpr, lr_tpr, label=lr_legend)
plt.plot(nb_fpr, nb_tpr, label=nb_legend)
# plt.plot(sv_fpr, sv_tpr, label=sv_legend)
plt.plot(mlp_fpr, mlp_tpr, label=mlp_legend)
plt.plot(rf_fpr, rf_tpr, label=rf_legend)

plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

### Observations
The grid search improved most models at least a fair amount. The Random Forest was the best, with K-Nearest Neighbors and Logistic Regression not far behind.